In [ ]:
# default_exp dataset

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

from os import listdir, makedirs
from os.path import join, isfile, isdir, dirname, normpath, basename, sep, splitext, split
from PIL import Image as PILImage
from functools import partial
from datetime import datetime
from logging.handlers import MemoryHandler
import numpy as np
import shutil
import json
import sys
import csv
import argparse
import logging

In [ ]:
# export

CATEGORY_LABEL_KEY = 'category'
IMAGE_SET_FOLDER = 'imagesets/segmentation/car_damage'
DEFAULT_CATEGORIES_FILE = 'categories.txt'
DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE = 'annotations.csv'
DEFAULT_SEGMENTATION_ANNOTATIONS_FILE = 'via_region_data.json'
DEFAULT_SPLIT = 0.2
DATA_SET_FOLDER = 'datasets'
SEMANTIC_MASK_FOLDER = 'semantic_masks'
IMAGE_EXTENSIONS = ['.jpg']
FOLDER_FILTER = ['.DS_Store']
IMAGES_TRAIN_VAL_FOLDER = 'trainval'
IMAGES_TEST_FOLDER = 'test'
DATA_SET_TRAIN_FOLDER = 'train'
DATA_SET_VAL_FOLDER = 'val'
DATA_SET_TEST_FOLDER = 'test'
IMAGE_SEGMENTATION = 'segmentation'
IMAGE_CLASSIFICATION = 'classification'
NOT_CATEGORIZED = '[NOT_CATEGORIZED]'

In [ ]:
# export

logger = logging.getLogger(__name__)

# Data-Set

> Data-Set Notes.

Creating a data-set for a classification or segmentation task. If an annotation file is present, the annotations are also prepared.
The data-set is created based on an image-set.

## Image-Set

Image-sets are collected images to build a data-set from, stored in the `imagesets` folder.
The `imagesets` folder contains the following folder structure:
- imagesets/*[image_set_type]*/*[image_set_name]*

Inside the `[image_set_name]` folder are the following files / folders
- `test/`: test images (benchmark)
- `trainval/`: training and validation images for [cross validation](https://pdc-pj.backlog.jp/wiki/RAD_RAD/Neural+Network+-+Training)
- `categories.txt`: all categories (classes) the image-set contains

## Categories (Classes)

Category names of an image-set or data-set are written in plain text, separated by a *new line* and stored in a file `categories.txt`.
The order of the category names is important and reflects the index of the result vector for easy mapping.

In [ ]:
# export


def read_categories(categories_file):
    """
    Reads the categories from a categories file.
    If the optional `categories_file` is not given, the file name *categories.txt* is used by default
    `path`: The folder, the categories file is stored in
    `categories_file`: the categories file name, if not the default
    return: a list of the category names
    """
    if categories_file is None:
        categories_file = DEFAULT_CATEGORIES_FILE

    with open(categories_file) as f:
        categories = f.read().strip().split('\n')
    return categories

## Data-Set Folders

Data-sets are stored in the `datasets` base folder.
The `datasets` folder contains the following folder structure:
- datasets/*[data_set_type]*/*[data_set_name]*
where `[data_set_type]` is the same as the corresponding `[image_set_type]` and `[data_set_name]` is the same as the corresponding `[image_set_name]`.

Inside the `[data_set_name]` folder are the following files / folders
- `test/`: test set (benchmark)
- `train/`: training set
- `val/`: validation set
- `categories.txt`: all categories (classes) the data-set contains

In [ ]:
# export


class DataSet:
    def __init__(self, name, base_path, image_set_path, categories_path):
        self.name = name
        self.base_path = base_path
        self.image_set_path = image_set_path
        self.categories_path = categories_path
        self.categories = read_categories(categories_path)
        self.type = basename(dirname(image_set_path))
        self.folder = None
        self.train_folder = None
        self.val_folder = None
        self.test_folder = None
        self.logger = self.get_logger()

    def create_folders(self):
        """
        Creates the data-set folder structure, if not exist
        """

        # create data-set folder and remove previous data if exist
        self.folder = create_folder(join(self.base_path, self.type, self.name), clear=True)
        self.logger.info("Created folder {}".format(self.folder))
        self.train_folder = create_folder(join(self.folder, DATA_SET_TRAIN_FOLDER))
        self.logger.info("Created folder {}".format(self.train_folder))
        self.val_folder = create_folder(join(self.folder, DATA_SET_VAL_FOLDER))
        self.logger.info("Created folder {}".format(self.val_folder))
        if isdir(join(self.image_set_path, DATA_SET_TEST_FOLDER)):
            self.test_folder = create_folder(join(self.folder, DATA_SET_TEST_FOLDER))
            self.logger.info("Created folder {}".format(self.test_folder))

    def validate(self):
        """
        Validates the images.
        """
        pass

    def copy(self, train_files, val_files, test_files=None):
        """
        Copy the images to the data-set
        `train_files`: The list of training images
        `val_files`: The list of validation images
        `test_files`: The list of test images
        """
        pass

    def build(self, split=DEFAULT_SPLIT, seed=None, sample=None):
        """
        Build the data-set. This is the main logic.
        This method validates the images against the annotations,
        split the image-set into train and val on given split percentage,
        creates the data-set folders and copies the image.
        If a sample percentage is given, a sub-set is created as sample.
        `split`: The percentage of images which will be copied into the validation set
        `seed`: A random seed to reproduce splits
        `sample`: The percentage of images from train, val and test which will also from a sample set
        """
        pass

    @classmethod
    def get_logger(cls):
        """
        Configures default logging for the system.
        """
        logger = logging.getLogger(__name__)
        logger.addHandler(logging.NullHandler())
        return logger

## Create a classification data-set

Classification data-set can be created from a classification image-set. 
All images are validated, if they belong to one of the given categories. If categories with no images are found or images belong to a category not listed in `categories.txt`, the data-set can not be created.

In [ ]:
# export


class ClassificationDataSet(DataSet):
    def __init__(self, name, base_path, image_set_path, categories_path, annotations_path=None):
        super().__init__(name, base_path, image_set_path, categories_path)
        self.annotations_path = annotations_path
        self.annotations = self._read_annotations(annotations_path) if annotations_path else []

    def validate(self):
        """
        Validates if all images of a classification image set belongs to a category in `categories.txt` as well as
        that each listet category contains at least one image.
        return: a tuple with a list of missing categories and a list of empty categories
        """

        # validate only the trainval images, the test images have no annotations to validate
        content_folder = join(self.image_set_path, IMAGES_TRAIN_VAL_FOLDER)

        self.logger.info('Start validate image set at {}'.format(content_folder))

        base_path = join(content_folder, '')

        files = list(map(lambda f: self.trim_base_path(f, base_path), scan_files(content_folder)))

        self.logger.info('Found {} files at {}'.format(len(files), content_folder))

        steps = [
            {
                'name': 'missing_categories',
                'choices': {
                    's': 'Skip',  # just skip the category
                    'S': 'Skip All',
                    'a': 'Add',  # add category to category index
                    'A': 'Add All',
                },
                'choice': None,
                'condition': lambda tags, cats: list(filter(lambda t: t not in cats, tags)),
                'message': '[{}] -> {} : {}Found categories not in the category index. Missing Categories: {}',
                'transform': lambda tags, cats: list(map(lambda t: cats.append(t) if t not in cats else None, tags)),
                'skip': lambda tags, cats: " ".join(filter(lambda t: t not in tags, cats)),
            },
            {
                'name': 'empty_categories',
                'choices': {
                    's': 'Skip',  # skip files with empty categories
                    'S': 'Skip All',
                    'k': 'Keep',  # keep files with empty categories
                    'K': 'Keep All',
                },
                'choice': None,
                'condition': lambda tags, _: not tags,
                'message': '[{}] -> {} : {}Has no category assigned.{}',
                'transform': lambda _, cats: cats,
                'skip': lambda tags, cats: None,
            }
        ]

        # if no annotation file exist, generate annotations based on the folder names
        if not self.annotations:
            self.logger.info("No annotation file found, Start try to detect categories from folder structure.")
            self.annotations = self._annotations_from_file_paths(self.categories, files)
            self.logger.info("Finished try to detect categories from folder structure.")

        handled_files = {}
        used_categories = set([])

        # skip the title column
        for ind, (file, tags) in enumerate(self.annotations[1:]):
            index = ind + 1  # need to count the title column
            filename = file + ".jpg"
            tag_list = tags.split(" ")
            # validate annotation
            for step in steps:
                step_condition = step['condition'](tag_list, self.categories)
                if step_condition:
                    message = step['message'].format(index, filename, ' ', ' , '.join(step_condition))
                    step['choice'] = input_feedback(message, step['choice'], step['choices'])

                    choice_op = step['choice'].lower()
                    # if skip
                    if choice_op == 's':
                        handled_files[index] = step['skip'](step_condition, tag_list)
                        message = step['message'].format(index, filename, '{} '.format(step['choices'][choice_op]),
                                                         ' , '.join(step_condition))
                        self.logger.info(message)
                    else:
                        step['transform'](step_condition, self.categories)
                        message = step['message'].format(index, filename, '{} '.format(step['choices'][choice_op]),
                                                         ' , '.join(step_condition))
                        self.logger.info(message)

            if tag_list:
                used_categories.update(tag_list)

            if filename not in files:
                self.logger.info('[{}] -> {} : Annotated file do not exist, skip annotation.'.format(index, filename))
                handled_files[index] = None

        empty_categories = frozenset(self.categories) - used_categories
        if empty_categories:
            self.logger.info('The following categories have no images: {}'.format(" , ".join(empty_categories)))

        for index, tags in handled_files.items():
            self.annotations[index][1] = tags

        self.annotations = list(filter(lambda a: a[1] is not None, self.annotations))

        self.logger.info('Finished validate image set at {}'.format(content_folder))

    def copy(self, train_files, val_files, test_files=None):
        """
        Copy the images to the data-set and generate the annotations for train and val images.
        `train_files`: The list of training images
        `val_files`: The list of validation images
        `test_files`: The list of test images
        """

        train_folder_name = basename(self.train_folder)
        val_folder_name = basename(self.val_folder)

        train_val_image_folder = join(self.image_set_path, IMAGES_TRAIN_VAL_FOLDER)
        test_image_folder = join(self.image_set_path, IMAGES_TEST_FOLDER)

        # copy the categories files
        shutil.copy2(self.categories_path, join(self.folder, DEFAULT_CATEGORIES_FILE))

        # generate the annotations to match the processed files
        # and add the header and append the is_valid column
        annotations = [[h for h in (self.annotations[0] + ['is_valid'])]]

        # copy train files
        for index, filename in train_files:
            # copy image
            sub_folder = split(filename)[0]
            target_folder = join(self.train_folder, sub_folder)
            if sub_folder:
                create_folder(target_folder)
            shutil.copy2(join(train_val_image_folder, filename + ".jpg"), target_folder)
            self.logger.info('Copied {}'.format(join(target_folder, filename + ".jpg")))
            annotation = [join(train_folder_name, filename)] + self.annotations[index][1:] + [False]
            annotations.append(annotation)

        # copy val files
        for index, filename in val_files:
            # copy image
            sub_folder = split(filename)[0]
            target_folder = join(self.val_folder, sub_folder)
            if sub_folder:
                create_folder(target_folder)
            shutil.copy2(join(train_val_image_folder, filename + ".jpg"), target_folder)
            self.logger.info('Copied {}'.format(join(target_folder, filename + ".jpg")))
            annotation = [join(val_folder_name, filename)] + self.annotations[index][1:] + [True]
            annotations.append(annotation)

        # write the modified annotation file into data-set folder
        annotation_path = join(self.folder, DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE)
        with open(annotation_path, mode='w') as csv_file:
            csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerows(annotations)
            self.logger.info('Wrote annotations file {}'.format(annotation_path))

        # copy test_files, if exist
        if test_files:
            for filename in test_files:
                shutil.copy2(join(test_image_folder, filename), self.test_folder)
                self.logger.info('Copied {}'.format(join(self.test_folder, filename)))

    def build(self, split=DEFAULT_SPLIT, seed=None, sample=None):
        """
        Build the data-set. This is the main logic.
        This method validates the images against the annotations,
        split the image-set into train and val on given split percentage,
        creates the data-set folders and copies the image.
        If a sample percentage is given, a sub-set is created as sample.
        `split`: The percentage of images which will be copied into the validation set
        `seed`: A random seed to reproduce splits
        `sample`: The percentage of images from train, val and test which will also from a sample set
        """
        # validate the image set
        self.validate()

        # sort filenames by assigned tags
        tags_to_files = {}

        for index, (filename, tags) in enumerate(self.annotations[1:]):
            if tags not in tags_to_files:
                tags_to_files[tags] = []
            tags_to_files[tags].append((index + 1, filename))

        # split category files into train & val and create the sample split, if set
        train_files = []
        val_files = []
        sample_train_files = []
        sample_val_files = []

        for tags, files in tags_to_files.items():
            train, val = split_train_val_data(files, split, seed)
            train_files.extend(train)
            val_files.extend(val)

            # if a sample data set should be created, create the splits
            if sample:
                _, sample_train = split_train_val_data(train, sample, seed)
                _, sample_val = split_train_val_data(val, sample, seed)
                sample_train_files.extend(sample_train)
                sample_val_files.extend(sample_val)

        # scan the test images if exist
        images_test_folder = join(self.image_set_path, IMAGES_TEST_FOLDER)
        test_files = list(map(lambda f: basename(f), scan_files(images_test_folder))) if self.test_folder else None
        _, sample_test_files = split_train_val_data(test_files, sample, seed) if test_files and sample else (None, None)

        # copy the files
        self.copy(train_files, val_files, test_files)

        if sample:
            # create the sample data-set
            sample_data_set = ClassificationDataSet("{}_sample".format(self.name), self.base_path, self.image_set_path,
                                                    self.categories_path)
            # assign the annotations
            sample_data_set.annotations = self.annotations
            # create the data set folders
            sample_data_set.create_folders()
            # copy the sample images
            sample_data_set.copy(sample_train_files, sample_val_files, sample_test_files)

    def _read_annotations(self, annotations_file):
        """
        Reads an annotation file
        `annotations_file`: the path to the annotation file to read
        return: the annotations
        """
        with open(annotations_file) as csv_file:
            annotations = list(csv.reader(csv_file))
            self.logger.info('Found {} annotations at {}'.format(len(annotations), annotations_file))

        return annotations

    def _detect_category_index_from_paths(self, categories, file_paths):
        for path in file_paths:
            path_split = normpath(path).lstrip(sep).split(sep)
            for category in categories:
                try:
                    index = path_split.index(category)
                    self.logger.info("{} -> Found category in folder naming at index {}.".format(path, index))
                    return index
                except ValueError:
                    continue
            self.logger.info("{} -> Could not find any category in folder naming. Try next file.".format(path))
        return -1

    def _annotations_from_file_paths(self, categories, file_paths):
        annotations = [["image_name", "tags"]]
        index = self._detect_category_index_from_paths(categories, file_paths)
        if index >= 0:
            for path in file_paths:
                path_split = normpath(path).lstrip(sep).split(sep)
                annotations.append([splitext(path)[0], path_split[index]])
        return annotations

    @classmethod
    def trim_base_path(cls, file_path, base_path):
        if file_path.startswith(base_path):
            file_path = file_path[len(base_path):]
        return file_path

## Create a segmentation data-set

Segmentation data-set can be created from a segmentation image-set.
All images are validated against the annotations, if they contain at least one annotation and that the annotation category belongs to one of the given categories. The annotations have to be in [VIA v1](http://www.robots.ox.ac.uk/~vgg/software/via/via-1.0.5.html) json format. Rectangle annotations are converted into polygon annotations for unique segment generation.

In [ ]:
# export


class SegmentationDataSet(DataSet):
    def __init__(self, name, base_path, image_set_path, categories_path, annotations_path=None):
        super().__init__(name, base_path, image_set_path, categories_path)
        self.annotations_path = annotations_path
        self.annotations = self._read_annotations(annotations_path) if annotations_path else {}
        self.semantic_mask_folder = None

    def validate(self):
        """
        Validates, that each file has corresponding segmentation annotation with at least one region is set.
        """

        # validate only the trainval images, the test images have no annotations to validate
        content_folder = join(self.image_set_path, IMAGES_TRAIN_VAL_FOLDER)

        # convert the annotations before doing validation
        self._convert_annotation()

        self.logger.info('Start validate image set at {}'.format(content_folder))

        files = scan_files(content_folder)

        self.logger.info('Found {} files at {}'.format(len(files), content_folder))

        delete_annotations = {}

        for annotation_index, annotation in self.annotations.items():
            file_name = annotation['filename']
            regions = annotation['regions']

            file_path = join(content_folder, file_name)

            delete_regions = {}
            for region_index, region in regions.items():
                ra = region['region_attributes']
                attribute_name = ra[CATEGORY_LABEL_KEY] if ra and CATEGORY_LABEL_KEY in ra else None
                annotation_valid = attribute_name is not None and attribute_name in self.categories
                if not annotation_valid:
                    message = '{} : Region {} with category {} is not in category list, skip category.'
                    self.logger.info(message.format(file_name, region_index, attribute_name))
                    delete_regions[region_index] = True

            # delete regions after iteration is finished
            for region_index in delete_regions.keys():
                del regions[region_index]

            # validate for empty region
            if not regions:
                self.logger.info('{} : Has empty regions, skip file.'.format(annotation_index))
                delete_annotations[annotation_index] = True

            if file_name and file_path in files:
                index = files.index(file_path)
                files.pop(index)
            else:
                self.logger.info('{} : File of annotations do not exist, skip annotations.'.format(annotation_index))
                delete_annotations[annotation_index] = True

        for file_index, file in enumerate(files):
            self.logger.info('[{}] -> {} : File has no annotations, skip file.'.format(file_index, file))

        # delete annotations after iteration is finished
        for annotation_index in delete_annotations.keys():
            del self.annotations[annotation_index]

        self.logger.info('Finished validate image set at {}'.format(content_folder))

    def create_folders(self):
        """
        Creates the data-set folder structure, if not exist
        """
        super().create_folders()

        # create semantic mask file folder and remove previous data if exist
        self.semantic_mask_folder = create_folder(join(self.folder, SEMANTIC_MASK_FOLDER), clear=True)
        self.logger.info("Created folder {}".format(self.semantic_mask_folder))

    def copy(self, train_files, val_files, test_files=None):
        """
        Copy the images to the data-set, generate the annotations for train and val images
        and generate the semantic masks.
        `train_files`: The list of training images
        `val_files`: The list of validation images
        `test_files`: The list of test images
        """

        self.logger.info('Start copy files from {} to {}'.format(self.image_set_path, self.folder))

        train_val_image_folder = join(self.image_set_path, IMAGES_TRAIN_VAL_FOLDER)
        test_image_folder = join(self.image_set_path, IMAGES_TEST_FOLDER)

        # copy the categories files
        self.logger.info('Copy file {} to {}'.format(self.categories_path, self.folder))
        shutil.copy2(self.categories_path, join(self.folder, DEFAULT_CATEGORIES_FILE))

        # save semantic annotations
        self._save_semantic_masks(train_files + val_files)

        annotations_train = {}
        annotations_val = {}

        num_files = len(train_files)
        self.logger.info('Start copy {} files to {}'.format(num_files, self.train_folder))

        for key in train_files:
            # copy image
            annotation = self.annotations[key]
            shutil.copy2(join(train_val_image_folder, annotation['filename']), self.train_folder)
            # add annotation
            annotations_train[key] = annotation
        self.logger.info('Finished copy {} files to {}'.format(num_files, self.train_folder))

        num_files = len(val_files)
        self.logger.info('Start copy {} files to {}'.format(num_files, self.val_folder))

        for key in val_files:
            # copy image
            annotation = self.annotations[key]
            shutil.copy2(join(train_val_image_folder, annotation['filename']), self.val_folder)
            # add annotation
            annotations_val[key] = annotation

        self.logger.info('Finished copy {} files to {}'.format(num_files, self.val_folder))

        # write the split train annotations
        if annotations_train:
            annotations_target_path = join(self.train_folder, DEFAULT_SEGMENTATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            with open(annotations_target_path, 'w') as outfile:
                json.dump(annotations_train, outfile)

        # write the split val annotations
        if annotations_val:
            annotations_target_path = join(self.val_folder, DEFAULT_SEGMENTATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            with open(annotations_target_path, 'w') as outfile:
                json.dump(annotations_val, outfile)

        # copy test_files, if exist
        if test_files:

            num_files = len(test_files)
            self.logger.info('Start copy {} files to {}'.format(num_files, self.test_folder))

            for filename in test_files:
                shutil.copy2(join(test_image_folder, filename), self.test_folder)

            self.logger.info('Finished copy {} files to {}'.format(num_files, self.test_folder))

        self.logger.info('Finished copy files from {} to {}'.format(self.image_set_path, self.folder))

    def build(self, split=DEFAULT_SPLIT, seed=None, sample=None):
        """
        Build the data-set. This is the main logic.
        This method validates the images against the annotations,
        split the image-set into train and val on given split percentage,
        creates the data-set folders and copies the image.
        A segmentation mask per image is also generated.
        If a sample percentage is given, a sub-set is created as sample.
        `split`: The percentage of images which will be copied into the validation set
        `seed`: A random seed to reproduce splits
        `sample`: The percentage of images from train, val and test which will also from a sample set
        """

        self.logger.info('Validation set contains {}% of the images.'.format(int(split * 100)))

        # validate the image set
        self.validate()

        # split category files into train & val and create the sample split, if set
        train_files, val_files = split_train_val_data(list(self.annotations.keys()), split, seed)
        sample_train_files = []
        sample_val_files = []

        # if a sample data set should be created, create the splits
        if sample:
            _, sample_train_files = split_train_val_data(train_files, sample, seed)
            _, sample_val_files = split_train_val_data(val_files, sample, seed)

        # scan the test images if exist
        images_test_folder = join(self.image_set_path, IMAGES_TEST_FOLDER)
        test_files = list(map(lambda f: basename(f), scan_files(images_test_folder))) if self.test_folder else None
        _, sample_test_files = split_train_val_data(test_files, sample, seed) if test_files and sample else (None, None)

        # copy the files
        self.copy(train_files, val_files, test_files)

        if sample:
            sample_name = "{}_sample".format(self.name)

            self.logger.info('Start build {} data-set containing {}% of images at {}'.format(sample_name,
                                                                                             int(sample * 100),
                                                                                             self.base_path))

            # create the sample data-set
            sample_data_set = SegmentationDataSet(sample_name, self.base_path, self.image_set_path,
                                                  self.categories_path)
            # assign the converted annotations
            sample_data_set.annotations = self.annotations

            # create the data set folders
            sample_data_set.create_folders()
            # copy the sample images
            sample_data_set.copy(sample_train_files, sample_val_files, sample_test_files)

            self.logger.info('Finished build {} data-set containing {}% of images at {}'.format(sample_name,
                                                                                                int(sample * 100),
                                                                                                self.base_path))

    def _convert_annotation(self):
        """
        Converts segmentation regions from rectangle to polygon, if exist
        """

        # only the trainval images have annotation, not the test images
        content_folder = join(self.image_set_path, IMAGES_TRAIN_VAL_FOLDER)

        steps = [
            {
                'name': 'position',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    't': 'Trim',  # transform the annotation
                    'T': 'Trim All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, size: p_min < 0 or p_max >= size,
                'message': '{} -> {} : {}Exceeds image {}. \n Points \n x: {} \n y: {}',
                'transform': lambda p, size=0: max(min(p, size - 1), 0),
            },
            {
                'name': 'size',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    'k': 'Keep',  # transform the annotation (in this case do nothing)
                    'K': 'Keep All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, _: p_max - p_min <= 1,
                'message': '{} -> {} : {}Shape {} is <= 1 pixel. \n Points \n x: {} \n y: {}',
                'transform': lambda p, size=0: p,
            }
        ]

        self.logger.info('Start convert image annotations from {}'.format(self.annotations_path))

        for annotation_index, annotation in self.annotations.items():
            file_name = annotation['filename']
            regions = annotation['regions']

            if not regions:
                continue

            with Image.open(join(content_folder, file_name)) as image:
                image_width, image_height = image.size

                delete_regions = {}
                for region_index, region in regions.items():
                    shape_attributes = region['shape_attributes']
                    shape_type = shape_attributes['name']

                    # convert from rect to polygon if needed
                    if shape_type == 'rect':
                        x = max(shape_attributes["x"], 0)
                        y = max(shape_attributes["y"], 0)
                        max_x = min(x + shape_attributes["width"], image_width - 1)
                        max_y = min(y + shape_attributes["height"], image_height - 1)
                        shape_attributes = {
                            "name": "polygon",
                            "all_points_x": [x, x, max_x, max_x, x],
                            "all_points_y": [y, max_y, max_y, y, y],
                        }
                        region['shape_attributes'] = shape_attributes

                    for step in steps:
                        # validate the shape size
                        x_min = min(shape_attributes['all_points_x'])
                        x_max = max(shape_attributes['all_points_x'])
                        y_min = min(shape_attributes['all_points_y'])
                        y_max = max(shape_attributes['all_points_y'])

                        width_condition = step['condition'](x_min, x_max, image_width)
                        height_condition = step['condition'](y_min, y_max, image_height)
                        if width_condition or height_condition:
                            size_message = ['width'] if width_condition else []
                            size_message.extend(['height'] if height_condition else [])
                            message = step['message'].format(annotation_index, region_index, ' ',
                                                             ' and '.join(size_message),
                                                             shape_attributes['all_points_x'],
                                                             shape_attributes['all_points_y'])

                            step['choice'] = input_feedback(message, step['choice'], step['choices'])

                            choice_op = step['choice'].lower()
                            # if skip the shapes
                            if choice_op == 's':
                                delete_regions[region_index] = True
                                message = step['message'].format(annotation_index, region_index,
                                                                 '{} '.format(step['choices'][choice_op]),
                                                                 ' and '.join(size_message),
                                                                 shape_attributes['all_points_x'],
                                                                 shape_attributes['all_points_y'])
                                self.logger.info(message)

                                break
                            else:
                                shape_attributes['all_points_x'] = list(map(partial(step['transform'],
                                                                                    size=image_width),
                                                                            shape_attributes['all_points_x']))
                                shape_attributes['all_points_y'] = list(map(partial(step['transform'],
                                                                                    size=image_height),
                                                                            shape_attributes['all_points_y']))

                                message = step['message'].format(annotation_index, region_index,
                                                                 '{} '.format(step['choices'][choice_op]),
                                                                 ' and '.join(size_message),
                                                                 shape_attributes['all_points_x'],
                                                                 shape_attributes['all_points_y'])
                                self.logger.info(message)

                # delete regions after iteration is finished
                for region_index in delete_regions.keys():
                    del regions[region_index]

        print('Finished convert image annotations from {}'.format(self.annotations_path))

    def _save_semantic_masks(self, keys):
        """
        Create semantic segmentation mask png files out of the annotations.
        The mask file name is the same as the image file name but is stored in png format.
        `keys`: The annotation keys to create the segmentation masks for
        """
        from skimage import draw

        num_masks = len(keys)
        self.logger.info('Start create {} segmentation masks in {}'.format(num_masks, self.semantic_mask_folder))

        # only the trainval images have annotation, not the test images
        content_folder = join(self.image_set_path, IMAGES_TRAIN_VAL_FOLDER)

        for index, key in enumerate(keys):
            annotation = self.annotations[key]
            file_name = annotation['filename']
            regions = annotation['regions']

            if not regions:
                continue

            with Image.open(join(content_folder, file_name)) as image:
                image_width, image_height = image.size

                # Convert polygons to a bitmap mask of shape
                # [height, width]
                mask = np.zeros((image_height, image_width), dtype=np.uint8)

                # sort the regions by category priority for handling pixels which are assigned to more than one category
                # the category with higher index overpaint the category with lower index
                for region in sorted(regions.values(),
                                     key=lambda r: self.categories.index(r['region_attributes'][CATEGORY_LABEL_KEY])):
                    shape_attributes = region['shape_attributes']
                    region_attributes = region['region_attributes']
                    category = region_attributes[CATEGORY_LABEL_KEY]
                    class_id = self.categories.index(category) + 1

                    # Get indexes of pixels inside the polygon and set them to 1
                    rr, cc = draw.polygon(shape_attributes['all_points_y'], shape_attributes['all_points_x'])
                    mask[rr, cc] = class_id

                # save the semantic mask
                im = Image.fromarray(mask)
                mask_path = join(self.semantic_mask_folder, splitext(file_name)[0] + '.png')
                im.save(mask_path)

                self.logger.info('{} / {} - Created segmentation mask {}'.format(index + 1, num_masks, mask_path))

        self.logger.info('Finish create {} segmentation masks in {}'.format(num_masks, self.semantic_mask_folder))

    def _read_annotations(self, annotations_file):
        """
        Reads an VIA annotation file
        `annotations_file`: the path to the annotation file to read
        return: the annotations
        """
        with open(annotations_file) as json_file:
            annotations = json.load(json_file)
            self.logger.info('Found {} annotations at {}'.format(len(annotations), annotations_file))

        return annotations

## Helper Methods

In [ ]:
# export


def scan_files(folder, file_extensions=None):
    """
    Scan the folder for files and filter files by file extension.
    If the optional `file_extension` is not set, **.jpg** as file extension is used by default.
    `folder`: the folder to scan for files
    `file_extensions`: the allowed file extensions
    return: the file path list
    """
    if file_extensions is None:
        file_extensions = IMAGE_EXTENSIONS

    results = []
    if isdir(folder):
        files = listdir(folder)
        for file in files:
            file_path = join(folder, file)
            filename, file_extension = splitext(file_path)
            if isfile(file_path) and (file_extension in file_extensions):
                results.append(file_path)
            elif isdir(file_path):
                results.extend(scan_files(file_path, file_extensions))
    return results

In [ ]:
# export


def scan_path(path):
    """
    Scan a folder and subfolders until the file level.
    `path`: the current directory to parse for folders
    return: a list of folders prefixed with base directory
    """
    has_sub_folders = False
    results = []
    names = listdir(path)
    for name in names:
        subfolder_path = join(path, name)
        if isdir(subfolder_path) and name not in FOLDER_FILTER:
            has_sub_folders = True
            results = results + scan_path(subfolder_path)
    if not has_sub_folders:
        results.append(path)
    return results

In [ ]:
# export


def delete_folder(path):
    """
    Deletes a Folder, if exist.
    `path`: the folder path including the folder name
    return: the created folder path
    """
    if isdir(path):
        shutil.rmtree(path)
    return path

In [ ]:
# export


def create_folder(path, clear=False):
    """
    Creates a Folder, if not exist.
    `path`: the folder path including the folder name
    `clear`: wherether or not the created folder should be empty
    return: the created folder path
    """

    if clear:
        # remove previous data-set folder, if exist to clear the contents
        delete_folder(path)

    if not isdir(path):
        makedirs(path)

    return path

In [ ]:
# export


def split_train_val_data(data, val_size=0.2, seed=None):
    """
    Splits the images in train and validation set
    `data`: the data to split
    `val_size`: the size of the validation set in percentage
    `seed`: A random seed to reproduce splits.
    return: the splittet train, validation images
    """

    from sklearn.model_selection import train_test_split
    train, test = train_test_split(data, random_state=seed, test_size=val_size) if len(data) > 1 else (data, [])
    return train, test

In [ ]:
# export


def input_feedback(msg, choice, choices):
    # if decision is already made for all contents, skip feedback
    if not (choice and choice.isupper()):
        prompt = '{} \n choices: {} '.format(msg, ', '.join(['{} ({})'.format(k, v) for k, v in choices.items()]))
        while True:
            choice = input(prompt)
            if choice in choices:
                break
    return choice


In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.
    """
    log_memory_handler = MemoryHandler(1, flushLevel=logging_level)
    log_memory_handler.setLevel(logging_level)

    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setLevel(logging_level)

    logger.addHandler(log_memory_handler)
    logger.addHandler(stdout_handler)

    logger.setLevel(logging_level)

    return log_memory_handler

## Build a data-set

To build a data-set from an image-set. Handles currently classification and segmentation image-sets taken from the image-set-type, the parent folder, the image-set folder is located in. 

In [ ]:
# export


def build_data_set(category_file_path, annotation_file_path, split, seed, sample, output):
    """
    Build the data-set for training, Validation and test
    `category_file_path`: the filename of the categories file
    `annotation_file_path`: the file path to the annotation file
    `split`: the size of the validation set as percentage
    `seed`: random seed to reproduce splits
    `sample`: the size of the sample set as percentage
    `output`: the dataset base folder to build the dataset in
    """
    log_memory_handler = configure_logging()

    logger.info('Build parameters:')
    logger.info(' '.join(sys.argv[1:]))
    logger.info('Build configuration:')
    logger.info('category_file_path: {}'.format(category_file_path))
    logger.info('annotation_file_path: {}'.format(annotation_file_path))
    logger.info('split: {}'.format(split))
    logger.info('seed: {}'.format(seed))
    logger.info('sample: {}'.format(sample))
    logger.info('output: {}'.format(output))

    path = dirname(category_file_path)
    data_set_type = basename(dirname(path))
    annotation_file_name = basename(annotation_file_path) if annotation_file_path else ''

    # set the data-set name
    data_set_name = splitext(annotation_file_name)[0]
    if annotation_file_name in ['', DEFAULT_SEGMENTATION_ANNOTATIONS_FILE, DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE]:
        data_set_name = basename(path)

    data_set = None
    logger.info('Start build {} data-set {} at {}'.format(data_set_type, data_set_name, output))

    if data_set_type == IMAGE_CLASSIFICATION:
        data_set = ClassificationDataSet(data_set_name, output, path, category_file_path, annotation_file_path)
    if data_set_type == IMAGE_SEGMENTATION:
        data_set = SegmentationDataSet(data_set_name, output, path, category_file_path, annotation_file_path)

    if data_set:
        # create the data set folders
        logger.info("Start create the data-set folders at {}".format(data_set.base_path))
        data_set.create_folders()
        logger.info("Finished create the data-set folders at {}".format(data_set.base_path))

        # create the build log file
        log_file_name = datetime.now().strftime('build_%Y.%m.%d-%H.%M.%S.log')
        file_handler = logging.FileHandler(join(data_set.folder, log_file_name), encoding="utf-8")
        log_memory_handler.setTarget(file_handler)

        # build the data set
        data_set.build(split, seed, sample)

    logger.info('Finished build {} data-set {} at {}'.format(data_set_type, data_set_name, output))

## Run from command line

To run the data-set builder from command line, use the following command:
`python -m car_damage_detection.dataset [parameters]`

The following parameters are supported:
- `[categories]`: The path to the categories file. (e.g.: *categories.txt*)
- `--annotation`: The path to the annotation file, the data-set is build from. (e.g.: *imagesets/classification/car_damage/annotations.csv* for classification, *imagesets/segmentation/car_damage/via_region_data.json* for segmentation)
- `--split`: The percentage of the data which belongs to validation set, default to *0.2* (=20%)
- `--seed`: A random seed to reproduce splits, default to None
- `--category-label-key`: The key, the category name can be found in the annotation file, default to *name*.
- `--sample`: The percentage of the data which will be copied as a sample set with in a separate folder with "_sample" suffix. If not set, no sample data-set will be created.
- `--output`: The path of the dataset folder, default to *../datasets*.

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    parser = argparse.ArgumentParser()
    parser.add_argument("categories",
                        help="The path to the image-set categories file.")
    parser.add_argument("--annotation",
                        help="The path to the image-set annotation file, the data-set is build from.",
                        default=None)
    parser.add_argument("--split",
                        help="Percentage of the data which belongs to validation set.",
                        type=float,
                        default=0.2)
    parser.add_argument("--seed",
                        help="A random seed to reproduce splits.",
                        type=int,
                        default=None)
    parser.add_argument("--category-label-key",
                        help="The key of the category name.",
                        default=CATEGORY_LABEL_KEY)
    parser.add_argument("--sample",
                        help="Percentage of the data which will be copied as a sample set.",
                        type=float,
                        default=0)
    parser.add_argument("--output",
                        help="The path of the data-set folder.",
                        default=DATA_SET_FOLDER)
    args = parser.parse_args()

    CATEGORY_LABEL_KEY = args.category_label_key

    build_data_set(args.categories, args.annotation, args.split, args.seed, args.sample, args.output)
